In [9]:
import requests

In [10]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
from elasticsearch import Elasticsearch

In [13]:
es_client = Elasticsearch('http://localhost:9200')

In [14]:
es_client.info()

ObjectApiResponse({'name': '1c1469dd8db9', 'cluster_name': 'docker-cluster', 'cluster_uuid': '95fbJix7S4CexGPFRTGYYw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "section": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}   

index_name = "homework_responses5"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework_responses5'})

In [17]:
print("Available indexes:")
for index in index_name:
    print(index)

Available indexes:
h
o
m
e
w
o
r
k
_
r
e
s
p
o
n
s
e
s
5


In [18]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 41.19it/s]


In [20]:
query = 'How do I execute a command in a running docker container?'

In [21]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [22]:
response = es_client.search(index=index_name, body=search_query)

In [23]:
response

ObjectApiResponse({'took': 723, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'homework_responses5', '_id': 'O1ceS5ABrtCsKLdl3VA8', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'homework_responses5', '_id': 'SVceS5ABrtCsKLdlrk4h', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a 

# Q4: Filtering

In [24]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [25]:
search_results = es_client.search(index=index_name, body=search_query)

In [26]:
search_results

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'homework_responses5', '_id': 'O1ceS5ABrtCsKLdl3VA8', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'homework_responses5', '_id': 'WlceS5ABrtCsKLdl4FAi', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:

# Q5: Tempalates

In [27]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [28]:
search_results['hits']['hits']

[{'_index': 'homework_responses5',
  '_id': 'O1ceS5ABrtCsKLdl3VA8',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'homework_responses5',
  '_id': 'WlceS5ABrtCsKLdl4FAi',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local

In [29]:
result_docs = []

for hit in search_results['hits']['hits']:
    result_docs.append(hit['_source'])

In [30]:
search_results

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'homework_responses5', '_id': 'O1ceS5ABrtCsKLdl3VA8', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'homework_responses5', '_id': 'WlceS5ABrtCsKLdl4FAi', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:

In [31]:
context = ""

# for doc in search_results:
#         context = context + context_template.format(question={doc['question']}, text={doc['text']}) + "\n\n"

for doc in search_results['hits']['hits']:
    question = doc['_source']['question']
    text = doc['_source']['text']
    context += context_template.format(question=question, text=text) + "\n\n"


In [32]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [33]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [34]:
prompt = prompt_template.format(question=query, context=context).strip()

In [35]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT: Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dire

In [36]:
len(prompt)

1462

## Q6

In [29]:
pip install tiktoken


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [32]:
print(encoding)

<Encoding 'o200k_base'>


In [33]:
encoded_prompt = encoding.encode(prompt)


In [34]:
num_tokens = len(encoded_prompt)

In [35]:
num_tokens

322

# 7. Extras using different models
## Google Gemini API

https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python

In [1]:
pip install -q -U google-generativeai



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Mistral

https://docs.mistral.ai/getting-started/quickstart/

In [2]:
pip install mistralai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="What is the best French cheese?")]
)

print(chat_response.choices[0].message.content)


France is renowned for its cheese, and there are hundreds of varieties to choose from, each with its unique taste, texture, and aroma. Therefore, the "best" French cheese can depend on personal preference. However, some of the most popular and acclaimed French cheeses include:

1. Comté: A semi-hard cheese made from unpasteurized cow's milk in the Franche-Comté region. It has a nutty, slightly sweet flavor.

2. Brie de Meaux: Often referred to as the "king of cheeses," this soft cheese from the Île-de-France region has a rich, buttery flavor and a creamy texture.

3. Roquefort: This is a tangy, crumbly blue cheese made from sheep's milk in the south of France.

4. Camembert: A soft, surface-ripened cow's milk cheese from Normandy. It has a strong, earthy flavor and a creamy interior.

5. Reblochon: A soft, rind-washed cheese from the Alps. It has a nutty, slightly fruity flavor and a creamy texture.

6. Chèvre: This is a general term for goat's cheese, which can come in many forms, fro

In [5]:
def llm(prompt):
    response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)]
    )
    
    return response.choices[0].message.content

In [37]:
llm(prompt)

"To execute a command in a running Docker container, you first need to find the container's ID. You can do this by running the command `docker ps`, which will list all running containers along with their IDs. Once you have the container's ID, you can execute a command in it by using the `docker exec` command followed by the `-it` flag, the container's ID, and the command you want to run. For example, if you want to start a bash shell in the container, you would use the command `docker exec -it <container-id> bash`."